# Mistralモデルでの構築

## はじめに

このレッスンでは以下を扱います：
- さまざまなMistralモデルの探索
- 各モデルのユースケースとシナリオの理解
- 各モデルの独自機能を示すコードサンプル


## Mistralモデル

このレッスンでは、3つの異なるMistralモデルを紹介します：  
**Mistral Large**、**Mistral Small**、および**Mistral Nemo**。

これらのモデルはすべてGithub Modelマーケットプレイスで無料で利用可能です。このノートブックのコードはこれらのモデルを使用して実行されます。Github Modelsを使用して[AIモデルでプロトタイプを作成する方法](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)の詳細はこちらをご覧ください。


## Mistral Large 2 (2407)
Mistral Large 2は現在Mistralのフラッグシップモデルであり、企業向けに設計されています。

このモデルは元のMistral Largeのアップグレード版であり、以下を提供します
- より大きなコンテキストウィンドウ - 128k 対 32k
- 数学およびコーディングタスクでのパフォーマンス向上 - 平均精度76.9% 対 60.4%
- 多言語パフォーマンスの向上 - 対応言語：英語、フランス語、ドイツ語、スペイン語、イタリア語、ポルトガル語、オランダ語、ロシア語、中国語、日本語、韓国語、アラビア語、ヒンディー語。

これらの特徴により、Mistral Largeは以下に優れています
- *Retrieval Augmented Generation (RAG)* - より大きなコンテキストウィンドウによる
- *Function Calling* - このモデルはネイティブの関数呼び出しを備えており、外部ツールやAPIとの統合が可能です。これらの呼び出しは並列または順次のいずれでも行うことができます。
- *Code Generation* - Python、Java、TypeScript、C++の生成に優れています。


### Mistral Large 2を使用したRAGの例


この例では、Mistral Large 2を使用してテキストドキュメントに対してRAGパターンを実行しています。質問は韓国語で書かれており、著者の大学入学前の活動について尋ねています。

Cohere Embeddings Modelを使用して、テキストドキュメントと質問の両方の埋め込みを作成します。このサンプルでは、ベクトルストアとしてfaiss Pythonパッケージを使用しています。

Mistralモデルに送信されるプロンプトには、質問と質問に類似した取得済みのチャンクの両方が含まれています。モデルはその後、自然言語で応答を提供します。


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Smallは、プレミアム/エンタープライズカテゴリに属するMistralファミリーの別のモデルです。名前が示す通り、このモデルはスモールランゲージモデル（SLM）です。Mistral Smallを使用する利点は以下の通りです：
- Mistral LargeやNeMoなどのMistral LLMと比較してコスト削減が可能 - 価格が80%低下
- 低レイテンシー - MistralのLLMと比べて応答が速い
- 柔軟性 - 必要なリソースに対する制約が少なく、さまざまな環境で展開可能

Mistral Smallは以下に最適です：
- 要約、感情分析、翻訳などのテキストベースのタスク
- コスト効果が高いため頻繁にリクエストが行われるアプリケーション
- レビューやコード提案などの低レイテンシーのコードタスク


## Mistral Small と Mistral Large の比較

Mistral Small と Large のレイテンシの違いを示すために、以下のセルを実行してください。

同じプロンプトに対して、3～5秒の応答時間の違いが見られるはずです。また、応答の長さやスタイルの違いにも注目してください。


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

このレッスンで取り上げた他の2つのモデルと比較して、Mistral NeMoはApache2ライセンスの唯一の無料モデルです。

これは、Mistralの以前のオープンソースLLMであるMistral 7Bのアップグレード版と見なされています。

NeMoモデルのその他の特徴は次のとおりです：

- *より効率的なトークナイゼーション:* このモデルは、より一般的に使用されているtiktokenの代わりにTekkenトークナイザーを使用しています。これにより、より多くの言語やコードでのパフォーマンスが向上します。

- *ファインチューニング:* ベースモデルはファインチューニング可能です。これにより、ファインチューニングが必要なユースケースでの柔軟性が高まります。

- *ネイティブ関数呼び出し* - Mistral Largeと同様に、このモデルは関数呼び出しのトレーニングを受けています。これにより、最初のオープンソースモデルの1つとしてユニークな存在となっています。


## Mistral NeMo

このレッスンで取り上げた他の2つのモデルと比較して、Mistral NeMoはApache2ライセンスの唯一の無料モデルです。

これは、Mistralの以前のオープンソースLLMであるMistral 7Bのアップグレード版と見なされています。

NeMoモデルのその他の特徴は次のとおりです：

- *より効率的なトークナイゼーション:* このモデルは、より一般的に使用されているtiktokenの代わりにTekkenトークナイザーを使用しています。これにより、より多くの言語やコードでのパフォーマンスが向上します。

- *ファインチューニング:* ベースモデルはファインチューニング可能です。これにより、ファインチューニングが必要なユースケースでの柔軟性が高まります。

- *ネイティブ関数呼び出し* - Mistral Largeと同様に、このモデルは関数呼び出しのトレーニングを受けています。これにより、最初のオープンソースモデルの一つとしてユニークな存在となっています。


### トークナイザーの比較

このサンプルでは、Mistral NeMo がトークン化をどのように処理するかを Mistral Large と比較して見ていきます。

両方のサンプルは同じプロンプトを使用していますが、NeMo の方が Mistral Large よりも少ないトークンを返すことがわかるはずです。


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## 学びはここで終わりません、旅を続けましょう

このレッスンを終えたら、[Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) をチェックして、生成AIの知識をさらにレベルアップしましょう！


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**：  
本書類はAI翻訳サービス「Co-op Translator」（https://github.com/Azure/co-op-translator）を使用して翻訳されました。正確性の向上に努めておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。原文の言語による文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や誤訳についても、当方は責任を負いかねます。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
